In [9]:
import pandas as pd
import numpy as np

In [11]:
import os
os.getcwd()

'C:\\Users\\mailb'

In [4]:

# GitHub raw URL (important: replace %20 with spaces or use raw link)
url = "https://raw.githubusercontent.com/artem-ai-blip/Data-Analysis-3/main/Assignment%202/Crete,%202024,%20Q1.csv"

# Load the dataset
crete_df = pd.read_csv(url)

# Display basic info
crete_df.info()
crete_df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25772 entries, 0 to 25771
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              25772 non-null  object 
 1   name                            21603 non-null  object 
 2   host_id                         21602 non-null  float64
 3   host_name                       21602 non-null  object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   21602 non-null  object 
 6   latitude                        21602 non-null  float64
 7   longitude                       21602 non-null  float64
 8   room_type                       21602 non-null  object 
 9   price                           21077 non-null  float64
 10  minimum_nights                  21602 non-null  float64
 11  number_of_reviews               21602 non-null  float64
 12  last_review                     

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license;
0,27966,Heraklion-Pinelopi Apartment,120502.0,Emmanouil,NaN,Μαλεβιζίου,35.33012,25.08012,Entire home/apt,45.0,2.0,128.0,2023-09-20,0.82,2.0,334.0,5.0,00000247117;
1,"29130,""Villa Kallergi - Athena, 12 guests"",125...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"29849,""Villa Kallergi - Nefeli, 6 guests"",1252...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,29856,Matala Dimitris Villa - Four Bed Room,128653.0,Dimitris,NaN,Φαιστού,34.99533,24.75654,Private room,148.0,1.0,44.0,2023-09-20,0.26,4.0,33.0,1.0,00001193488;
4,31023,Chryssoula Guesthouse balcony (200mbps),133208.0,Chryssoula,NaN,Χανίων,35.51439,24.01793,Entire home/apt,61.0,2.0,313.0,2024-06-23,2.15,4.0,204.0,12.0,1246760;


neighbourhood_group has only nulls — we’ll likely drop this.

price has ~2,695 missing values.

last_review and reviews_per_month are missing for about 1/4 of the data — we can impute or engineer features from that.

license; has a semicolon in its name — we should fix that column name.

In [6]:
# Clean column names
crete_df.columns = crete_df.columns.str.strip().str.replace(';', '', regex=False)

# Drop columns with all missing values
crete_df = crete_df.drop(columns=['neighbourhood_group'])

# Check missing data
missing = crete_df.isnull().sum()
missing = missing[missing > 0]
print("Missing values:\n", missing)

# Basic imputation strategy
crete_df['price'] = crete_df['price'].fillna(crete_df['price'].median())
crete_df['reviews_per_month'] = crete_df['reviews_per_month'].fillna(0)
crete_df['last_review'] = pd.to_datetime(crete_df['last_review'], errors='coerce')
crete_df['last_review'] = crete_df['last_review'].fillna(pd.Timestamp('2023-01-01'))


Missing values:
 name                              4169
host_id                           4170
host_name                         4170
neighbourhood                     4170
latitude                          4170
longitude                         4170
room_type                         4170
price                             4695
minimum_nights                    4170
number_of_reviews                 4170
last_review                       9871
reviews_per_month                 9871
calculated_host_listings_count    4170
availability_365                  4170
number_of_reviews_ltm             4170
license                           4170
dtype: int64


In [7]:
# Drop unnecessary columns
crete_df = crete_df.drop(columns=['name', 'host_id', 'host_name', 'license'])

# Drop rows with missing values in key columns
crete_df = crete_df.dropna(subset=[
    'neighbourhood', 'latitude', 'longitude', 'room_type',
    'minimum_nights', 'number_of_reviews', 'calculated_host_listings_count',
    'availability_365', 'number_of_reviews_ltm'
])

# Confirm cleanup
crete_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21602 entries, 0 to 25771
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              21602 non-null  object        
 1   neighbourhood                   21602 non-null  object        
 2   latitude                        21602 non-null  float64       
 3   longitude                       21602 non-null  float64       
 4   room_type                       21602 non-null  object        
 5   price                           21602 non-null  float64       
 6   minimum_nights                  21602 non-null  float64       
 7   number_of_reviews               21602 non-null  float64       
 8   last_review                     21602 non-null  datetime64[ns]
 9   reviews_per_month               21602 non-null  float64       
 10  calculated_host_listings_count  21602 non-null  float64       
 11  av

Amendments to the data 

1. Date-Based Features
Extract year/month/day from last_review

Create a "days since last review" feature

2. Log-Transform Skewed Columns
price, minimum_nights, number_of_reviews, reviews_per_month, etc.

3. Categorical Encoding
room_type → One-hot or ordinal encoding

neighbourhood → One-hot (or frequency encoding if too many categories)

4. Interaction Features (Optional)
reviews_per_month * availability_365

number_of_reviews_ltm / number_of_reviews

In [10]:
# 1. Date-based feature
crete_df['days_since_last_review'] = (pd.Timestamp('2024-01-01') - crete_df['last_review']).dt.days

# 2. Log-transform skewed variables
for col in ['price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month']:
    crete_df[f'log_{col}'] = np.log1p(crete_df[col])

# 3. One-hot encode categorical variables
crete_df = pd.get_dummies(crete_df, columns=['room_type', 'neighbourhood'], drop_first=True)

# Final shape check
print(f"Final dataset shape: {crete_df.shape}")
crete_df.head()


Final dataset shape: (21602, 42)


,id,latitude,longitude,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,...,neighbourhood_Μινώα Πεδιάδας,neighbourhood_Μυλοποτάμου,neighbourhood_Οροπεδίου Λασιθίου,neighbourhood_Πλατανιά,neighbourhood_Ρεθύμνης,neighbourhood_Σητείας,neighbourhood_Σφακίων,neighbourhood_Φαιστού,neighbourhood_Χανίων,neighbourhood_Χερσονήσου
0,27966,35.330120,25.080120,45.0,2.0,128.0,2023-09-20,0.82,2.0,334.0,...,0,0,0,0,0,0,0,0,0,0
3,29856,34.995330,24.756540,148.0,1.0,44.0,2023-09-20,0.26,4.0,33.0,...,0,0,0,0,0,0,0,1,0,0
4,31023,35.514390,24.017930,61.0,2.0,313.0,2024-06-23,2.15,4.0,204.0,...,0,0,0,0,0,0,0,0,1,0
5,31789,35.496480,23.696480,200.0,3.0,2.0,2012-09-23,0.01,1.0,365.0,...,0,0,0,0,0,0,0,0,0,0
6,34280,35.396159,25.025414,85.0,2.0,131.0,2023-09-27,0.97,11.0,31.0,...,0,0,0,0,0,0,0,0,0,0
